# Create Zarr of 9 years of daily CSSP China data [1851-1859]

In [1]:
import iris
import os
import sys
import logging
import xarray as xr
import numpy as np

import crd_utils as crd
import umdates_utils as um

from datetime import datetime, timedelta

## Create a list of all the files we want to process

In [2]:
# daily data filenames
filepath = '/data/cssp-china/mini-dataset-24-01-19/20CR/daily'
files = sorted(os.listdir(filepath))
print(len(files))

5850


In [3]:
# generate all possible filenames for the time period
runid = 'apepd'
startd = datetime(1851, 1, 1)   # 00Z on Jan 01 1851
endd = datetime(1860, 1, 1)     # 00Z on Jan 01 1860
freq = 'pa'

decade_filenames = um.UMFileList(runid, startd, endd, freq)
print(len(decade_filenames))

3288


In [4]:
# find the overlap of actual filenames with all possible filenames in that decade
filenames = list(set(files).intersection(set(decade_filenames)))
filenames.sort()
print(len(filenames))

329


In [5]:
filepaths = [os.path.join(filepath, filename) for filename in filenames]
print(len(filepaths))

329


## Set all the Cube, Dataset and Zarr variables we need to process the cubes

In [6]:
%%time
cubelist = iris.load(filepaths[0:1])
cubelist[7]

CPU times: user 1.51 s, sys: 123 ms, total: 1.63 s
Wall time: 2.28 s


<iris 'Cube' of geopotential_height / (m) (time: 4; pressure: 14; grid_latitude: 219; grid_longitude: 286)>

In [7]:
print(cubelist)

0: m01s05i217 / (unknown)              (time: 4; grid_latitude: 219; grid_longitude: 286)
1: air_pressure_at_sea_level / (Pa)    (time: 4; grid_latitude: 219; grid_longitude: 286)
2: air_temperature / (K)               (time: 4; grid_latitude: 219; grid_longitude: 286)
3: air_temperature / (K)               (time: 4; grid_latitude: 219; grid_longitude: 286)
4: air_temperature / (K)               (time: 4; grid_latitude: 219; grid_longitude: 286)
5: air_temperature / (K)               (time: 4; pressure: 14; grid_latitude: 219; grid_longitude: 286)
6: cloud_area_fraction / (1)           (time: 4; grid_latitude: 219; grid_longitude: 286)
7: geopotential_height / (m)           (time: 4; pressure: 14; grid_latitude: 219; grid_longitude: 286)
8: precipitation_flux / (kg m-2 s-1)   (time: 4; grid_latitude: 219; grid_longitude: 286)
9: relative_humidity / (%)             (time: 4; grid_latitude: 219; grid_longitude: 286)
10: relative_humidity / (%)             (time: 4; grid_latitude: 219; gr

In [8]:
CUBENAMES = [cube.name() for cube in cubelist]
UNIQUE_COORDS = crd.unique_coords_list(cubelist)
COORD_NAME_MAPPING = crd.get_new_coord_names(UNIQUE_COORDS)

In [9]:
print([coord.name() for coord in UNIQUE_COORDS])

['time', 'grid_latitude', 'grid_longitude', 'forecast_reference_time', 'forecast_period', 'height', 'pressure', 'grid_latitude', 'grid_longitude', 'height']


In [10]:
COORD_NAME_MAPPING[1]

('grid_latitude_1', 'grid_longitude_1', 'height_1')

In [11]:
ZARR = '/data/cssp-china/zarr_daily_1851-1859'
CHUNKS = {'time': 200, 'pressure':1, 'grid_latitude': 219, 'grid_longitude': 286, 'grid_latitude_1': 218, 'grid_longitude_1': 286}

## Initialise logging

In [12]:
LOGFILE = '/data/cssp-china/zarr_append_log.log'
logging.basicConfig(filename=LOGFILE,
                    level=logging.DEBUG,
                    format='%(asctime)s %(message)s',
                    datefmt='%d/%m/%Y %H:%M:%S')

In [13]:
if not os.path.isfile(LOGFILE):
    os.mknod(LOGFILE)

if os.stat(LOGFILE).st_size == 0:
    logging.info('Initiate log')
    print(f'Log initiated at {LOGFILE}')

Log initiated at /data/cssp-china/zarr_append_log.log


## Create a Zarr with 21 cubes, which definitely amounts to 200 time steps (~20 days)

In [ ]:
%%time
cubelist0 = iris.load(filepaths[0:21])
cubelist0[7]

In [ ]:
%%time
crd.rename_cubes(cubelist0, CUBENAMES, COORD_NAME_MAPPING, dryrun=False)

In [ ]:
%%time
dalist0 = crd.cubelist_to_dalist(cubelist0)
ds0 = xr.merge(dalist0)

In [ ]:
ds0

In [ ]:
%%time
logging.info(f'Creating {ZARR}')
crd.ds_to_zarr(ds0, ZARR, chunks=CHUNKS)

In [ ]:
dsz0 = xr.open_zarr(ZARR)
dsz0

In [ ]:
# Size of Dataset in GB
dsz0.nbytes / 1e9

In [ ]:
logging.info(f'  Successfull creation of {ZARR}')
logging.info(f'  Processed filenames 0:{filenames[0]} - 21:{filenames[21]}')
logging.info(f'  Chunking {CHUNKS}')

## ~Append a couple of cubes without rechunking~

In [ ]:
# %%time
# cubelist1 = iris.load(filepaths[21:24])
# cubelist1[0]

In [ ]:
# %%time
# crd.rename_cubes(cubelist1, CUBENAMES, COORD_NAME_MAPPING, dryrun=False, verbose=False)
# dalist1 = crd.cubelist_to_dalist(cubelist1)
# ds1 = xr.merge(dalist1)
# ds1

In [ ]:
# %%time
# crd.ds_to_zarr(ds1, ZARR, chunks=CHUNKS)

In [ ]:
# dsz1 = xr.open_zarr(zarr)
# dsz1

## Now loop through the remaining cubes and append to the Zarr we created

In [ ]:
def logprint(message):
    logging.info(message)
    print(message)

In [ ]:
# Loop through in bunches of 12 cubes, so as to not use to much memory at one time
start = 21
step = 5
# stop = len(filepaths)
stop = start+(2*step)

logprint(f'Appending data to {ZARR}')

for i in range(start, stop, step):
    logprint(f'  Processing filenames {i}:{filenames[i]} - {i+step}:{filenames[i+step]}')
    try:
        cubelist = iris.load(filepaths[i:i+step])
        logprint(f'  | Loaded files successfully')
        
        crd.rename_cubes(cubelist, CUBENAMES, COORD_NAME_MAPPING, dryrun=False, verbose=False)
        logprint(f'  | Renamed files successfully')
        
        dalist = crd.cubelist_to_dalist(cubelist)
        ds = xr.merge(dalist)
        logprint(f'  | Created dataset successfully')
        
        crd.ds_to_zarr(ds, ZARR, chunks=CHUNKS)
        logprint(f'  | Appended to Zarr {ZARR} successfully')
        
    except Exception as e:
        logprint(f'  X ERROR: {e}')
        raise e

In [ ]:
step = 12
start = 21
print(len(list(range(start, len(filepaths), step))))
display(list(range(start, len(filepaths), step)))